# Linear Models

Linear models make a prediction using a linear function of the input features $X$ 

$$f_{\mathbf{w}}(\mathbf{x}) = \sum_{i=1}^{p} w_i \cdot x_i + w_{0}$$

Learn $w$ from $X$, given a loss function $\mathcal{L}$:

$$\underset{\mathbf{w}}{\operatorname{argmin}} \mathcal{L}(f_\mathbf{w}(X))$$

* Many algorithms with different $\mathcal{L}$: Least squares, Ridge, Lasso, Logistic Regression, Linear SVMs,...
* Can be very powerful (and fast), especially for large datasets with many features.
* Can be generalized to learn non-linear patterns: _Generalized Linear Models_
    * Features can be augmentented with polynomials of the original features
    * Features can be transformed according to a distribution (Poisson, Tweedie, Gamma,...)
    * Some linear models (e.g. SVMs) can be _kernelized_ to learn non-linear functions

## 1. Linear models for regression

### 1.1. Linear Regression (aka Ordinary Least Squares)
* Loss function is the _sum of squared errors_ (SSE) (or residuals) between predictions $\hat{y}_i$ (red) and the true regression targets $y_i$ (blue) on the training set.

$$\mathcal{L}_{SSE} = \sum_{n=1}^{N} (y_n-\hat{y}_n)^2 = \sum_{n=1}^{N} (y_n-(\mathbf{w}\mathbf{x_n} + w_0))^2$$ 

<img src="https://raw.githubusercontent.com/ML-course/master/master/notebooks/images/02_least_squares.png" alt="ml" style="margin: 0 auto; width: 400px;"/>

#### 1.1.1. Solving ordinary least squares
* Convex optimization problem with unique closed-form solution:
    
    $$w^{*} = (X^{T}X)^{-1} X^T Y$$
    
    * Add a column of 1's to the front of X to get $w_0$
    * Slow. Time complexity is quadratic in number of features: $\mathcal{O}(p^2n)$
        * X has $n$ rows, $p$ features, hence $X^{T}X$ has dimensionality $p \cdot p$
    * **Only works if $n>p$**
* **Very easily overfits**.
    * coefficients $w$ become very large (steep incline/decline)
    * small change in the input *x* results in a very different output *y* 
    * No hyperparameters that control model complexity

#### 1.1.2. Gradient Descent
* Start with an initial, random set of weights: $\mathbf{w}^0$
* Given a differentiable loss function $\mathcal{L}$ (e.g. $\mathcal{L}_{SSE}$), compute $\nabla \mathcal{L}$
* For least squares: $\frac{\partial \mathcal{L}_{SSE}}{\partial w_i}(\mathbf{w}) = -2\sum_{n=1}^{N} (y_n-\hat{y}_n) x_{n,i}$
    * If feature $X_{:,i}$ is associated with big errors, the gradient wrt $w_i$ will be large
* Update _all_ weights slightly (by _step size_ or _learning rate_ $\eta$) in 'downhill' direction.
* Basic _update rule_ (step s): 

    $$\mathbf{w}^{s+1} = \mathbf{w}^s-\eta\nabla \mathcal{L}(\mathbf{w}^s)$$

<img src="https://raw.githubusercontent.com/ML-course/master/master/notebooks/images/01_gradient_descent.jpg" alt="ml" style="width: 400px;"/>

* Important hyperparameters
    * Learning rate
        * Too small: slow convergence. Too large: possible divergence
    * Maximum number of iterations
        * Too small: no convergence. Too large: wastes resources
    * Learning rate decay with decay rate $k$
        * E.g. exponential ($\eta^{s+1} = \eta^{0}  e^{-ks}$), inverse-time ($\eta^{s+1} = \frac{\eta^{s}}{1+ks}$),...
    * Many more advanced ways to control learning rate (see later)
        * Adaptive techniques: depend on how much loss improved in previous step

#### 1.1.3. Stochastic Gradient Descent (SGD)
* Compute gradients not on the entire dataset, but on a single data point $i$ at a time
    * Gradient descent: $\mathbf{w}^{s+1} = \mathbf{w}^s-\eta\nabla \mathcal{L}(\mathbf{w}^s) = \mathbf{w}^s-\frac{\eta}{n} \sum_{i=1}^{n} \nabla \mathcal{L_i}(\mathbf{w}^s)$
    * Stochastic Gradient Descent: $\mathbf{w}^{s+1} = \mathbf{w}^s-\eta\nabla \mathcal{L_i}(\mathbf{w}^s)$
* Many smoother variants, e.g.
    * Minibatch SGD: compute gradient on batches of data: $\mathbf{w}^{s+1} = \mathbf{w}^s-\frac{\eta}{B} \sum_{i=1}^{B} \nabla \mathcal{L_i}(\mathbf{w}^s)$
    * Stochastic Average Gradient Descent ([SAG](https://link.springer.com/content/pdf/10.1007/s10107-016-1030-6.pdf), [SAGA](https://proceedings.neurips.cc/paper/2014/file/ede7e2b6d13a41ddf9f4bdef84fdc737-Paper.pdf)). With $i_s \in [1,n]$ randomly chosen per iteration:
        * Incremental gradient: $\mathbf{w}^{s+1} = \mathbf{w}^s-\frac{\eta}{n} \sum_{i=1}^{n} v_i^s$ with $v_i^s = \begin{cases}\nabla \mathcal{L_i}(\mathbf{w}^s) & i = i_s \\ v_i^{s-1} & \text{otherwise} \end{cases}$
        
<img src="https://raw.githubusercontent.com/ML-course/master/master/notebooks/images/08_SGD.png" alt="ml" style="float: left; width: 400px;"/>

In [1]:
from sklearn.linear_model import SGDRegressor

model = SGDRegressor(loss="squared_loss")

### 1.2. Ridge regression

* Adds a penalty term to the least squares loss function:

$$\mathcal{L}_{Ridge} = \sum_{n=1}^{N} (y_n-(\mathbf{w}\mathbf{x_n} + w_0))^2 + \alpha \sum_{i=1}^{p} w_i^2$$ 

* Model is penalized if it uses large coefficients ($w$)
    * Each feature should have as little effect on the outcome as possible 
    * We don't want to penalize $w_0$, so we leave it out
* Regularization: explicitly restrict a model to avoid overfitting. 
    * Called L2 regularization because it uses the L2 norm: $\sum w_i^2$
* The strength of the regularization can be controlled with the $\alpha$ hyperparameter.
    * Increasing $\alpha$ causes more regularization (or shrinkage). Default is 1.0.
- Still convex. Can be optimized in different ways:
    - **Closed form solution** (a.k.a. Cholesky): $w^{*} = (X^{T}X + \alpha I)^{-1} X^T Y$
    - Gradient descent and variants, e.g. Stochastic Average Gradient (SAG,SAGA)
        - Conjugate gradient (CG): each new gradient is influenced by previous ones
    - **Use Cholesky for smaller datasets, Gradient descent for larger ones**

``` python
from sklearn.linear_model import Ridge
lr = Ridge(alpha=1.0).fit(X_train, y_train)
```

### 1.3. Lasso (Least Absolute Shrinkage and Selection Operator)
* Adds a different penalty term to the least squares sum:

$$\mathcal{L}_{Lasso} = \sum_{n=1}^{N} (y_n-(\mathbf{w}\mathbf{x_n} + w_0))^2 + \alpha \sum_{i=1}^{p} |w_i|$$

- Called L1 regularization because it uses the L1 norm, 可以用来做**特征选择/解决自变量间的多重共线性问题(变量之间存在强相关性, e.g. x_1 = 2x_2)**.
    - **Will cause many weights to be exactly 0(sparse models)**
    - Some features are ignored entirely: automatic feature selection
- Same parameter $\alpha$ to control the strength of regularization. 
    * Will again have a 'sweet spot' depending on the data
- **No closed-form solution**
- Convex, but no longer strictly convex, and not differentiable
    * Weights can be optimized using **_coordinate descent_**

#### 1.3.1. Coordinate descent (坐标下降法)
- Alternative for gradient descent, supports **non-differentiable convex loss functions** (e.g. $\mathcal{L}_{Lasso}$)
- In every iteration, optimize a single coordinate $w_i$ (find minimum in direction of $x_i$)
    - Continue with another coordinate, using a selection rule (e.g. round robin)
- Faster iterations. **No need to choose a step size (learning rate)**.
- May converge more slowly. Can't be parallellized.

<img src="https://raw.githubusercontent.com/ML-course/master/master/notebooks/images/02_cd.png" alt="ml" style="width: 400px;"/>

#### 1.3.2. Coordinate descent with Lasso

- Remember that $\mathcal{L}_{Lasso} = \mathcal{L}_{SSE} + \alpha \sum_{i=1}^{p} |w_i|$ 
- For one $w_i$: $\mathcal{L}_{Lasso}(w_i) = \mathcal{L}_{SSE}(w_i) + \alpha |w_i|$
- The L1 term is not differentiable but convex: we can compute the [_subgradient_](https://towardsdatascience.com/unboxing-lasso-regularization-with-proximal-gradient-method-ista-iterative-soft-thresholding-b0797f05f8ea) 
    - Unique at points where $\mathcal{L}$ is differentiable, a range of all possible slopes [a,b] where it is not
    - For $|w_i|$, the subgradient $\partial_{w_i} |w_i|$ =  $\begin{cases}-1 & w_i<0\\ [-1,1] & w_i=0 \\ 1 & w_i>0 \\ \end{cases}$

    - Subdifferential $\partial(f+g) = \partial f + \partial g$ if $f$ and $g$ are both convex
- To find the optimum for Lasso $w_i^{*}$, solve

    $$\begin{aligned} \partial_{w_i} \mathcal{L}_{Lasso}(w_i) &= \partial_{w_i} \mathcal{L}_{SSE}(w_i) + \partial_{w_i} \alpha |w_i| \\ 0 &= (w_i - \rho_i) + \alpha \cdot \partial_{w_i} |w_i| \\ w_i &= \rho_i - \alpha \cdot \partial_{w_i} |w_i| \end{aligned}$$

    - In which $\rho_i$ is the part of $\partial_{w_i} \mathcal{L}_{SSE}(w_i)$ excluding $w_i$ (assume $z_i=1$ for now)
        - $\rho_i$ can be seen as the $\mathcal{L}_{SSE}$ 'solution': $w_i = \rho_i$ if $\partial_{w_i} \mathcal{L}_{SSE}(w_i) = 0$ 
  $$\partial_{w_i} \mathcal{L}_{SSE}(w_i) = \partial_{w_i} \sum_{n=1}^{N} (y_n-(\mathbf{w}\mathbf{x_n} + w_0))^2 = z_i w_i -\rho_i $$ 

- We found: $w_i = \rho_i - \alpha \cdot \partial_{w_i} |w_i|$
- [The Lasso solution](https://xavierbourretsicotte.github.io/lasso_derivation.html) has the form of a _soft thresholding function_ $S$

    $$w_i^* = S(\rho_i,\alpha) = \begin{cases} \rho_i + \alpha, & \rho_i < -\alpha \\  0, & -\alpha < \rho_i < \alpha \\ \rho_i - \alpha, & \rho_i > \alpha \\ \end{cases}$$
    
    - Small weights (all weights between $-\alpha$ and $\alpha$) **become 0: sparseness!**
    - If the data is not normalized, $w_i^* = \frac{1}{z_i}S(\rho_i,\alpha)$ with constant $z_i = \sum_{n=1}^{N} x_{ni}^2$
- Ridge solution: $w_i = \rho_i - \alpha \cdot \partial_{w_i} w_i^2 = \rho_i - 2\alpha \cdot w_i$, thus $w_i^* = \frac{\rho_i}{1 + 2\alpha}$

- **参考1**: [凸优化笔记16：次梯度](https://zhuanlan.zhihu.com/p/139083837)
- **参考2**: [LASSO的坐标下降法求解](https://zhuanlan.zhihu.com/p/429541451)

### 1.4. Elastic-Net

* Adds both L1 and L2 regularization:

$$\mathcal{L}_{Elastic} = \sum_{n=1}^{N} (y_n-(\mathbf{w}\mathbf{x_n} + w_0))^2 + \alpha \rho \sum_{i=1}^{p} |w_i| + \alpha (1 -  \rho) \sum_{i=1}^{p} w_i^2$$ 

* $\rho$ is the L1 ratio
    * With $\rho=1$, $\mathcal{L}_{Elastic} = \mathcal{L}_{Lasso}$
    * With $\rho=0$, $\mathcal{L}_{Elastic} = \mathcal{L}_{Ridge}$
    * $0 < \rho < 1$ sets a trade-off between L1 and L2.
* Allows learning sparse models (like Lasso) while maintaining L2 regularization benefits
    * E.g. if 2 features are correlated, Lasso likely picks one randomly, Elastic-Net keeps both 
* Weights can be optimized using coordinate descent (similar to Lasso) 

### 1.5. Other Regression Loss

- **Huber loss:** switches from squared loss to linear loss past a value $\epsilon$
    - More robust against **outliers**
- **Epsilon insensitive:** ignores errors smaller than $\epsilon$, and linear past that
    - Aims to fit function so that residuals are at most $\epsilon$
    - **Also known as Support Vector Regression (`SVR` in sklearn)**
- Squared Epsilon insensitive: ignores errors smaller than $\epsilon$, and squared past that


## 2. Linear models for Classification

Aims to find a hyperplane that separates the examples of each class.  
For binary classification (2 classes), we aim to fit the following function: 

$\hat{y} = w_1 * x_1 + w_2 * x_2 +... + w_p * x_p + w_0 > 0$  
    
When $\hat{y}<0$, predict class -1, otherwise predict class +1

<p>

There are many algorithms for linear classification, differing in loss function, regularization techniques, and optimization method

* Most common techniques:
    * Convert target classes {neg,pos} to {0,1} and treat as a regression task
        * Logistic regression (Log loss)
        * Ridge Classification (Least Squares + L2 loss)
    * Find hyperplane that maximizes the margin between classes
        * Linear Support Vector Machines (Hinge loss)
    * Neural networks without activation functions
        * Perceptron (Perceptron loss)
    * SGDClassifier: can act like any of these by choosing loss function
        * Hinge, Log, Modified_huber, Squared_hinge, Perceptron

### 2.1. Logistic regression
* Aims to predict the _probability_ that a point belongs to the positive class
* Converts target values {negative, positive} to {0,1}
* Fits a _logistic_ (or _sigmoid_ or _S_ curve) function through these points
    * Maps (-Inf,Inf) to a probability [0,1]
    
    $$ \hat{y} = \textrm{logistic}(f_{\theta}(\mathbf{x})) = \frac{1}{1+e^{-f_{\theta}(\mathbf{x})}} $$
    
* E.g. in 1D: $ \textrm{logistic}(x_1w_1+w_0) = \frac{1}{1+e^{-x_1w_1-w_0}} $

#### 2.1.1. Loss function: Cross-entropy
* Models that return class probabilities can use _cross-entropy loss_
    
    $$\mathcal{L_{log}}(\mathbf{w}) = \sum_{n=1}^{N} H(p_n,q_n) = - \sum_{n=1}^{N} \sum_{c=1}^{C} p_{n,c} log(q_{n,c}) $$
    
    * Also known as log loss, logistic loss, or maximum likelihood
    - Based on true probabilities $p$ (0 or 1) and predicted probabilities $q$ over $N$ instances and $C$ classes
        - **Binary case (C=2)**: $\mathcal{L_{log}}(\mathbf{w}) = - \sum_{n=1}^{N} \big[ y_n log(\hat{y}_n) + (1-y_n) log(1-\hat{y}_n) \big]$
    * Penalty (a.k.a. 'surprise') grows exponentially as difference between $p$ and $q$ increases
        * If you are sure of an answer (high $q$) and it's wrong (low $p$), you definitely want to learn
    * Often used together with L2 (or L1) loss: $\mathcal{L_{log}}'(\mathbf{w}) = \mathcal{L_{log}}(\mathbf{w}) + \alpha \sum_{i} w_i^2 $

#### 2.1.2. Optimization methods (solvers) for cross-entropy loss
* Gradient descent (only supports L2 regularization)
    - Log loss is differentiable, so we can use (stochastic) gradient descent
    - Variants thereof, e.g. Stochastic Average Gradient (SAG, SAGA)
* Coordinate descent (supports both L1 and L2 regularization)
    - Faster iteration, but may converge more slowly, has issues with saddlepoints
    - Called `liblinear` in sklearn. **Can't run in parallel.**
* Newton-Rhapson or Newton Conjugate Gradient (only L2):
    - Uses the Hessian $H = \big[\frac{\partial^2 \mathcal{L}}{\partial x_i \partial x_j} \big]$: $\mathbf{w}^{s+1} = \mathbf{w}^s-\eta H^{-1}(\mathbf{w}^s) \nabla \mathcal{L}(\mathbf{w}^s)$
    - **Slow** for large datasets. **Works well if solution space is (near) convex**
* Quasi-Newton methods (only L2)
    - Approximate, faster to compute
    - E.g. Limited-memory Broyden–Fletcher–Goldfarb–Shanno (`lbfgs`)
        - Default in sklearn for Logistic Regression
- Data scaling helps convergence, minimizes differences between solvers

#### 2.1.3. In practice
* Logistic regression can also be found in `sklearn.linear_model`.
    * `C` hyperparameter is the _inverse_ regularization strength: $C=\alpha^{-1}$
    * `penalty`: type of regularization: L1, L2 (default), Elastic-Net, or None
    * `solver`: newton-cg, lbfgs (default), liblinear, sag, saga
* Increasing C: less regularization, tries to overfit individual points

``` python
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1).fit(X_train, y_train)
```

### 2.2. Support vector machines
- Decision boundaries close to training points may generalize badly
    - Very similar (nearby) test point are classified as the other class
- Choose a boundary that is as far away from training points as possible
- The __support vectors__ are the training samples closest to the hyperplane
- The __margin__ is the distance between the separating hyperplane and the _support vectors_
- Hence, our objective is to _maximize the margin_

参考1: [DDA3020 SVM Notes](../DDA3020/L07_SVM.pdf)

<p>

参考2: [Lecture 2. Linear models](https://ml-course.github.io/master/notebooks/02%20-%20Linear%20Models.html#kernelization)
### 2.3. Perceptron
